In [1]:
import os.path
import datetime

import numpy  as np
import pandas as pd

In [2]:
# Variables: dirs and files

dirBase   = '..'

dirData   = os.path.join(dirBase,'Data')
dirPicle  = os.path.join(dirBase,'Data','Pickle')
#dirTemp   = os.path.join(dirBase,'Temp','Temp','Class3-1')
#dirTemp   = os.path.join(dirBase,'Temp','Temp','Class3-2')
dirTemp   = os.path.join(dirBase,'Temp','Temp','Class3-3')
dirResult = os.path.join(dirBase,'Result','Leak') 

fileTrain      = os.path.join(dirData,'trainExt.csv')
fileValidation = os.path.join(dirData,'trainExtMin.csv')
fileTest       = os.path.join(dirData,'testExt.csv')

In [3]:
# Constants

chunkSize = 4000000
nClusters = 100
nTop      = 5


In [4]:
# rules field

rule1  = ['user_location_city','orig_destination_distance']
rule2  = ['srch_destination_id','hotel_country','hotel_market']
rule3  = ['srch_destination_id']
rule3b = ['srch_destination_id','dt0m']
rule4  = ['hotel_country']

ruleXtest    = ['cnt','is_booking']
ruleXtrain   = ['hotel_cluster'] + ruleXtest


ruleTrain    = rule1+rule2+[i for i in rule3b if i not in rule2]+ruleXtrain+['dt0y','dty']
ruleTest     = rule1+rule2+[i for i in rule3b if i not in rule2]+ruleXtest+['id']
ruleValidate = ruleTrain

In [5]:
def rule (data,list) :
    df       = data[list].groupby(list).count().reset_index()
    df['nn'] = df.index
    ar       = np.zeros((df.shape[0],nClusters),dtype=np.int16)
    return([df,ar])

In [6]:
# Reader for test data (chunks) on one time if not in disk
print(datetime.datetime.now(),'Read test datas ')
OK =    os.path.exists(os.path.join(dirTemp,'rule1Test0Empty.pkl')) and \
        os.path.exists(os.path.join(dirTemp,'rule2Test0Empty.pkl')) and \
        os.path.exists(os.path.join(dirTemp,'rule3Test0Empty.pkl')) and \
        os.path.exists(os.path.join(dirTemp,'rule3bTest0Empty.pkl')) and \
        os.path.exists(os.path.join(dirTemp,'rule4Test0Empty.pkl')) and \
        os.path.exists(os.path.join(dirTemp,'rule1Test1Empty.pkl.npy')) and \
        os.path.exists(os.path.join(dirTemp,'rule2Test1Empty.pkl.npy')) and \
        os.path.exists(os.path.join(dirTemp,'rule3Test1Empty.pkl.npy')) and \
        os.path.exists(os.path.join(dirTemp,'rule3bTest1Empty.pkl.npy')) and \
        os.path.exists(os.path.join(dirTemp,'rule4Test1Empty.pkl.npy'))

if not OK :

    readerTest = pd.read_csv(fileTest,chunksize=3000000,usecols=ruleTest);
    i = 0;
    for chunk in readerTest :
        i += 1; print(datetime.datetime.now(),'chunk :',i,chunk.shape)

        rule1Test = rule(chunk,rule1)
        rule2Test = rule(chunk,rule2)
        rule3Test = rule(chunk,rule3)
        rule3bTest= rule(chunk,rule3b)
        rule4Test = rule(chunk,rule4)

    del chunk
    del readerTest
    
    rule1Test[0].to_pickle(os.path.join(dirTemp,'rule1Test0Empty.pkl'))
    rule2Test[0].to_pickle(os.path.join(dirTemp,'rule2Test0Empty.pkl'))
    rule3Test[0].to_pickle(os.path.join(dirTemp,'rule3Test0Empty.pkl'))
    rule3bTest[0].to_pickle(os.path.join(dirTemp,'rule3bTest0Empty.pkl'))
    rule4Test[0].to_pickle(os.path.join(dirTemp,'rule4Test0Empty.pkl'))

    np.save(os.path.join(dirTemp,'rule1Test1Empty.pkl.npy'),rule1Test[1])
    np.save(os.path.join(dirTemp,'rule2Test1Empty.pkl.npy'),rule2Test[1])
    np.save(os.path.join(dirTemp,'rule3Test1Empty.pkl.npy'),rule3Test[1])
    np.save(os.path.join(dirTemp,'rule3bTest1Empty.pkl.npy'),rule3bTest[1])
    np.save(os.path.join(dirTemp,'rule4Test1Empty.pkl.npy'),rule4Test[1])
    
else :
    rule1df = pd.read_pickle(os.path.join(dirTemp,'rule1Test0Empty.pkl'))
    rule2df = pd.read_pickle(os.path.join(dirTemp,'rule2Test0Empty.pkl'))
    rule3df = pd.read_pickle(os.path.join(dirTemp,'rule3Test0Empty.pkl'))
    rule3bdf= pd.read_pickle(os.path.join(dirTemp,'rule3bTest0Empty.pkl'))
    rule4df = pd.read_pickle(os.path.join(dirTemp,'rule4Test0Empty.pkl'))
    
    rule1ar = np.load(os.path.join(dirTemp,'rule1Test1Empty.pkl.npy'))
    rule2ar = np.load(os.path.join(dirTemp,'rule2Test1Empty.pkl.npy'))
    rule3ar = np.load(os.path.join(dirTemp,'rule3Test1Empty.pkl.npy'))
    rule3bar= np.load(os.path.join(dirTemp,'rule3bTest1Empty.pkl.npy'))
    rule4ar = np.load(os.path.join(dirTemp,'rule4Test1Empty.pkl.npy'))
    
    rule1Test = [rule1df,rule1ar]
    rule2Test = [rule2df,rule2ar]
    rule3Test = [rule3df,rule3ar]
    rule3bTest= [rule3bdf,rule3bar]
    rule4Test = [rule4df,rule4ar]
    del rule1df,rule2df,rule3df,rule3bdf,rule4df
    del rule1ar,rule2ar,rule3ar,rule3bar,rule4ar
    
    print(rule1Test[1].shape,rule2Test[1].shape,rule3Test[1].shape,rule4Test[1].shape)

print(datetime.datetime.now(),'Done ')    

2016-06-05 16:14:49.871324 Read test datas 
(1385522, 100) (43831, 100) (40718, 100) (206, 100)
2016-06-05 16:14:57.819651 Done 


In [13]:
iGroup = 0
def ruleApply (group,Arr) :
    global iGroup
    nn = group['nn'].iloc[0]; 
    if (iGroup%50000==0) : 
        print(datetime.datetime.now(),'iGroup =',iGroup,'nn =',nn, 'len(group) = ',group.shape)
    iGroup  += 1
    gr  = group[['hotel_cluster','cnt']].groupby(['hotel_cluster']).sum().reset_index()
    Arr[nn][gr['hotel_cluster']] += gr['cnt']
    return True

In [14]:
def findTop (a) :
    b = pd.Series(a); b=np.array(b[b>0].nlargest(nTop).index); 
    b=b+1; b.resize(nTop); b=b-1 
    return(b)

In [15]:
# Reader for train data (chunks) if not in files

# ------------------------------------------------------------------------------

print(datetime.datetime.now(),'Read test datas from pickle')

rule1df = pd.read_pickle(os.path.join(dirTemp,'rule1Test0Empty.pkl'))
rule2df = pd.read_pickle(os.path.join(dirTemp,'rule2Test0Empty.pkl'))
rule3df = pd.read_pickle(os.path.join(dirTemp,'rule3Test0Empty.pkl'))
rule3bdf= pd.read_pickle(os.path.join(dirTemp,'rule3bTest0Empty.pkl'))
rule4df = pd.read_pickle(os.path.join(dirTemp,'rule4Test0Empty.pkl'))

rule1ar = np.load(os.path.join(dirTemp,'rule1Test1Empty.pkl.npy'))
rule2ar = np.load(os.path.join(dirTemp,'rule2Test1Empty.pkl.npy'))
rule3ar = np.load(os.path.join(dirTemp,'rule3Test1Empty.pkl.npy'))
rule3bar= np.load(os.path.join(dirTemp,'rule3bTest1Empty.pkl.npy'))
rule4ar = np.load(os.path.join(dirTemp,'rule4Test1Empty.pkl.npy'))

rule1Test = [rule1df,rule1ar]
rule2Test = [rule2df,rule2ar]
rule3Test = [rule3df,rule3ar]
rule3bTest= [rule3bdf,rule3bar]
rule4Test = [rule4df,rule4ar]
del rule1df,rule2df,rule3df,rule3bdf,rule4df
del rule1ar,rule2ar,rule3ar,rule3bar,rule4ar
# ------------------------------------------------------------------------------    
print(datetime.datetime.now(),'Read&Work train datas ')
chunkSize = 5000000

OK =    os.path.exists(os.path.join(dirTemp,'rule1Test1.pkl.npy')) and \
        os.path.exists(os.path.join(dirTemp,'rule2Test1.pkl.npy')) and \
        os.path.exists(os.path.join(dirTemp,'rule3Test1.pkl.npy')) and \
        os.path.exists(os.path.join(dirTemp,'rule3bTest1.pkl.npy')) and \
        os.path.exists(os.path.join(dirTemp,'rule4Test1.pkl.npy'))

if not OK :        
    readerTrain = pd.read_csv(fileTrain,chunksize=chunkSize,usecols=ruleTrain+['user_id']);
    readerTrain

    ichunk = -1;
    for chunk in readerTrain :
        ichunk += 1; 
        #if (i>1) : print(datetime.datetime.now(),'break :',i); break;
        print(datetime.datetime.now(),'chunk :',ichunk,chunk.shape)
        
        # Estimate influence for other user groups
        if True :
            chunk   = chunk.merge(grps20l,how='inner',on=['user_id'])
            print(datetime.datetime.now(),'after :',ichunk,chunk.shape)
        
        #
        # app1 = 3+is_booking*17
        # app2 = 1+is_booking*5
        #
        
        # Rule1 --> user_location_city+orig_destination_distance += 1
        print(datetime.datetime.now(),'1'); iGroup = 0;
        grp = chunk[rule1+ruleXtrain].groupby(rule1+['hotel_cluster']).sum().reset_index()
        grp.loc[:,'cnt'] = 1 # grp.cnt*(3+17*grp.is_booking)
        print(datetime.datetime.now(),'grp=(train group)',grp.shape)
        print(datetime.datetime.now(),'grpx=(test group)',rule1Test[0].shape)
        grp = grp.merge(rule1Test[0],how='inner',on=rule1,copy=False).groupby(rule1)
        print(datetime.datetime.now(),'grp=(new groups)',len(grp));
        grp.apply(ruleApply,rule1Test[1])
        #ruleWork(grp,rule1Test)
        

        # Rule2 --> (year==2014) : srch_destination_id+hotel_country+hotel_market += app1
        print(datetime.datetime.now(),'2'); iGroup = 0;
        grp = chunk.query('dty==2014')[rule2+ruleXtrain];
        grp.loc[:,'cnt'] = (3+17*grp.is_booking) #*grp.cnt
        grp = grp.groupby(rule2+['hotel_cluster']).sum().reset_index()
        print(datetime.datetime.now(),'grp=(train group)',grp.shape)
        print(datetime.datetime.now(),'grpx=(test group)',rule2Test[0].shape)
        grp = grp.merge(rule2Test[0],how='inner',on=rule2,copy=False).groupby(rule2)
        print(datetime.datetime.now(),'grp=(new groups)',len(grp));
        grp.apply(ruleApply,rule2Test[1])
        #ruleWork(grp,rule2Test,b0Add=3,b1Add=20)
        
        # Rule3 --> srch_destination_id += app1
        print(datetime.datetime.now(),'3'); iGroup = 0;
        grp = chunk[rule3+ruleXtrain]
        #grp.loc[:,'cnt'] = grp.cnt*(3+17*grp.is_booking)
        grp.loc[:,'cnt'] = (3+17*grp.is_booking) #*grp.cnt
        grp = grp.groupby(rule3+['hotel_cluster']).sum().reset_index()
        print(datetime.datetime.now(),'grp=(train group)',grp.shape)
        print(datetime.datetime.now(),'grpx=(test group)',rule3Test[0].shape)
        grp = grp.merge(rule3Test[0],how='inner',on=rule3).groupby(rule3)
        print(datetime.datetime.now(),'grp=(new groups)',len(grp))
        grp.apply(ruleApply,rule3Test[1])
        
        # Rule3b --> srch_destination_id+dt0m += app1
        print(datetime.datetime.now(),'3b'); iGroup = 0;
        grp = chunk[rule3b+ruleXtrain]
        #grp.loc[:,'cnt'] = grp.cnt*(3+17*grp.is_booking)
        #grp.loc[:,'cnt'] = (3+17*grp.is_booking) #*grp.cnt
        # cnt=cnt 
        grp = grp.groupby(rule3b+['hotel_cluster']).sum().reset_index()
        print(datetime.datetime.now(),'grp=(train group)',grp.shape)
        print(datetime.datetime.now(),'grpx=(test group)',rule3bTest[0].shape)
        grp = grp.merge(rule3bTest[0],how='inner',on=rule3b).groupby(rule3b)
        print(datetime.datetime.now(),'grp=(new groups)',len(grp))
        grp.apply(ruleApply,rule3bTest[1])
        
        # Rule4 --> hotel_country += app2
        print(datetime.datetime.now(),'4'); iGroup = 0;
        grp = chunk[rule4+ruleXtrain]
        grp.loc[:,'cnt'] = (1+5*grp.is_booking) # *grp.cnt
        grp = grp.groupby(rule4+['hotel_cluster']).sum().reset_index()
        print(datetime.datetime.now(),'grp=(train group)',grp.shape)
        print(datetime.datetime.now(),'grpx=(test group)',rule4Test[0].shape)
        grp = grp.merge(rule4Test[0],how='inner',on=rule4).groupby(rule4)
        print(datetime.datetime.now(),'grp=(new groups)',len(grp))
        grp.apply(ruleApply,rule4Test[1])
        
    del readerTrain
    np.save(os.path.join(dirTemp,'rule1Test1.pkl.npy'),rule1Test[1])
    np.save(os.path.join(dirTemp,'rule2Test1.pkl.npy'),rule2Test[1])
    np.save(os.path.join(dirTemp,'rule3Test1.pkl.npy'),rule3Test[1])
    np.save(os.path.join(dirTemp,'rule3bTest1.pkl.npy'),rule3bTest[1])
    np.save(os.path.join(dirTemp,'rule4Test1.pkl.npy'),rule4Test[1])
    
    print(datetime.datetime.now(),'Build array5 begin')
    rule1Arr5 = np.apply_along_axis(findTop,1,rule1Test[1]); print(rule1Arr5.shape)
    rule2Arr5 = np.apply_along_axis(findTop,1,rule2Test[1]); print(rule2Arr5.shape)
    rule3Arr5 = np.apply_along_axis(findTop,1,rule3Test[1]); print(rule3Arr5.shape)
    rule3bArr5= np.apply_along_axis(findTop,1,rule3bTest[1]); print(rule3bArr5.shape)
    rule4Arr5 = np.apply_along_axis(findTop,1,rule4Test[1]); print(rule4Arr5.shape)

    np.save(os.path.join(dirTemp,'rule1Test2.pkl.npy'),rule1Arr5)
    np.save(os.path.join(dirTemp,'rule2Test2.pkl.npy'),rule2Arr5)
    np.save(os.path.join(dirTemp,'rule3Test2.pkl.npy'),rule3Arr5)
    np.save(os.path.join(dirTemp,'rule3bTest2.pkl.npy'),rule3bArr5)
    np.save(os.path.join(dirTemp,'rule4Test2.pkl.npy'),rule4Arr5)

    print(datetime.datetime.now(),'Build array5 end')
    
else :  
    rule1ar = np.load(os.path.join(dirTemp,'rule1Test1.pkl.npy'))
    rule2ar = np.load(os.path.join(dirTemp,'rule2Test1.pkl.npy'))
    rule3ar = np.load(os.path.join(dirTemp,'rule3Test1.pkl.npy'))
    rule3bar= np.load(os.path.join(dirTemp,'rule3bTest1.pkl.npy'))
    rule4ar = np.load(os.path.join(dirTemp,'rule4Test1.pkl.npy'))
    
    rule1Test[1] = rule1ar
    rule2Test[1] = rule2ar
    rule3Test[1] = rule3ar
    rule3bTest[1] = rule3bar
    rule4Test[1] = rule4ar
    del rule1ar,rule2ar,rule3ar,rule3bar,rule4ar
    
    print(rule1Test[1].shape,rule2Test[1].shape,rule3Test[1].shape,rule4Test[1].shape)
    
    rule1Arr5 = np.load(os.path.join(dirTemp,'rule1Test2.pkl.npy'))
    rule2Arr5 = np.load(os.path.join(dirTemp,'rule2Test2.pkl.npy'))
    rule3Arr5 = np.load(os.path.join(dirTemp,'rule3Test2.pkl.npy'))
    rule3bArr5= np.load(os.path.join(dirTemp,'rule3bTest2.pkl.npy'))
    rule4Arr5 = np.load(os.path.join(dirTemp,'rule4Test2.pkl.npy'))
    
# Top clusters
rule5Arr5 = np.array([91,42,59,28])
        
print(datetime.datetime.now(),'Done ')

2016-06-05 16:20:17.043862 Read test datas from pickle
2016-06-05 16:20:25.799403 Read&Work train datas 
2016-06-05 16:20:38.641076 chunk : 0 (5000000, 12)
2016-06-05 16:20:40.817127 after : 0 (4619903, 12)
2016-06-05 16:20:40.817127 1
2016-06-05 16:20:45.613283 grp=(train group) (1720455, 5)
2016-06-05 16:20:45.613283 grpx=(test group) (1385522, 3)
2016-06-05 16:20:52.147468 grp=(new groups) 215750
2016-06-05 16:20:55.086864 iGroup = 0 nn = 33 len(group) =  (1, 6)


C:\Anaconda3-64\lib\site-packages\ipykernel\__main__.py:9: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


2016-06-05 16:23:48.299615 iGroup = 50000 nn = 324158 len(group) =  (1, 6)
2016-06-05 16:27:14.263733 iGroup = 100000 nn = 632194 len(group) =  (1, 6)
2016-06-05 16:30:38.757997 iGroup = 150000 nn = 961141 len(group) =  (1, 6)
2016-06-05 16:34:02.796195 iGroup = 200000 nn = 1270241 len(group) =  (1, 6)
2016-06-05 16:35:08.132112 2
2016-06-05 16:35:10.163136 grp=(train group) (165192, 6)
2016-06-05 16:35:10.163136 grpx=(test group) (43831, 4)
2016-06-05 16:35:10.241260 grp=(new groups) 23475
2016-06-05 16:35:10.970249 iGroup = 0 nn = 1 len(group) =  (19, 7)
2016-06-05 16:36:48.747924 3


C:\Anaconda3-64\lib\site-packages\pandas\core\indexing.py:545: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item_labels[indexer[info_axis]]] = value


2016-06-05 16:36:50.145873 grp=(train group) (187081, 4)
2016-06-05 16:36:50.145873 grpx=(test group) (40718, 2)
2016-06-05 16:36:50.192756 grp=(new groups) 24023
2016-06-05 16:36:50.352688 iGroup = 0 nn = 1 len(group) =  (22, 5)
2016-06-05 16:38:31.057831 3b
2016-06-05 16:38:32.989127 grp=(train group) (675852, 5)
2016-06-05 16:38:32.989127 grpx=(test group) (187210, 3)
2016-06-05 16:38:33.197657 grp=(new groups) 91305
2016-06-05 16:38:35.446800 iGroup = 0 nn = 1 len(group) =  (3, 6)
2016-06-05 16:42:04.251515 iGroup = 50000 nn = 92203 len(group) =  (3, 6)
2016-06-05 16:44:54.713918 4
2016-06-05 16:44:55.778783 grp=(train group) (6381, 4)
2016-06-05 16:44:55.778783 grpx=(test group) (206, 2)
2016-06-05 16:44:55.784785 grp=(new groups) 203
2016-06-05 16:44:55.791792 iGroup = 0 nn = 0 len(group) =  (42, 5)
2016-06-05 16:45:09.255650 chunk : 1 (5000000, 12)
2016-06-05 16:45:10.863567 after : 1 (4629399, 12)
2016-06-05 16:45:10.863567 1
2016-06-05 16:45:15.160729 grp=(train group) (170413

In [218]:
# Calculate EHR rules Rule1
if False :
    
    print(datetime.datetime.now(),'EHR rules ')

    chunkSize   = 10000000
    rule11      = ['user_id','user_location_city','srch_destination_id','hotel_market','hotel_country']
    rule11Xtrain= rule11 + ruleXtrain
    readerTrain = pd.read_csv(fileTrain,chunksize=chunkSize,usecols=rule11Xtrain);
    
    grps   = pd.DataFrame()

    ichunk = -1;
    for chunk in readerTrain :
        ichunk += 1; 
        #if (ichunk>1) : print(datetime.datetime.now(),'break :',ichunk); break;
        print(datetime.datetime.now(),'chunk :',ichunk,chunk.shape)
        
        #
        # app1 = 3+is_booking*17
        # app2 = 1+is_booking*5
        #
        
        # Rule1 --> user_location_city+orig_destination_distance += 1
        print(datetime.datetime.now(),'11'); iGroup = 0;
        
        grp = chunk[(chunk.is_booking>=1) & chunk.srch_destination_id.notnull() & chunk.hotel_market.notnull() & chunk.hotel_cluster.notnull()]
        
        grps = grps.append(grp)
        
    #grps = grps.groupby('user_id').sum().reset_index()
    print(grps.shape)    
    
    print(datetime.datetime.now(),'Done ')


2016-06-05 04:10:49.676898 User stats 
2016-06-05 04:11:18.640001 chunk : 0 (10000000, 8)
2016-06-05 04:11:18.640001 11
2016-06-05 04:12:19.153787 chunk : 1 (10000000, 8)
2016-06-05 04:12:19.153787 11
2016-06-05 04:12:55.648885 chunk : 2 (10000000, 8)
2016-06-05 04:12:55.648885 11
2016-06-05 04:13:17.935272 chunk : 3 (7670293, 8)
2016-06-05 04:13:17.935272 11
(3000693, 8)
2016-06-05 04:13:18.452991 Done 


In [222]:
grps.head()
grpss = grps.groupby(['user_id','user_location_city','srch_destination_id','hotel_market','hotel_country'])
len(grpss)

2578334

In [223]:
rule1Test[1].shape

(1385522, 100)

In [ ]:
# Calculate EHR rules Rule2
if False :
    
    print(datetime.datetime.now(),'EHR rules ')

    chunkSize   = 10000000
    rule12      = ['user_id','srch_destination_id','hotel_market','hotel_country']
    rule12Xtrain= rule12 + ruleXtrain
    readerTrain = pd.read_csv(fileTrain,chunksize=chunkSize,usecols=rule11Xtrain);
    
    grps   = pd.DataFrame()

    ichunk = -1;
    for chunk in readerTrain :
        ichunk += 1; 
        #if (ichunk>1) : print(datetime.datetime.now(),'break :',ichunk); break;
        print(datetime.datetime.now(),'chunk :',ichunk,chunk.shape)
        
        #
        # app1 = 3+is_booking*17
        # app2 = 1+is_booking*5
        #
        
        # Rule1 --> user_location_city+orig_destination_distance += 1
        print(datetime.datetime.now(),'11'); iGroup = 0;
        
        grp = chunk[(chunk.is_booking>=1) & chunk.srch_destination_id.notnull() & chunk.hotel_market.notnull() & chunk.hotel_cluster.notnull()]
        
        grps = grps.append(grp)
        
    #grps = grps.groupby('user_id').sum().reset_index()
    print(grps.shape)    
    
    print(datetime.datetime.now(),'Done ')


In [7]:
# Calculate user_id stats
if True :
    
    print(datetime.datetime.now(),'User stats ')

    chunkSize = 10000000
    readerTrain = pd.read_csv(fileTrain,chunksize=chunkSize,usecols=['user_id','is_booking','cnt','dt0y','dt0m']);
    readerTrain
    
    grps   = pd.DataFrame()

    ichunk = -1;
    for chunk in readerTrain :
        ichunk += 1; 
        #if (ichunk>1) : print(datetime.datetime.now(),'break :',ichunk); break;
        print(datetime.datetime.now(),'chunk :',ichunk,chunk.shape)
        
        #
        # app1 = 3+is_booking*17
        # app2 = 1+is_booking*5
        #
        
        # Rule1 --> user_location_city+orig_destination_distance += 1
        print(datetime.datetime.now(),'1'); iGroup = 0;
        grp = chunk[['user_id','is_booking','cnt']].groupby('user_id').sum().reset_index()
        
        grps = grps.append(grp)
        
    grps = grps.groupby('user_id').sum().reset_index()
    print(grps.shape)    
    
    print(datetime.datetime.now(),'Done ')


2016-06-05 16:15:45.070508 User stats 
2016-06-05 16:16:06.045301 chunk : 0 (10000000, 5)
2016-06-05 16:16:06.045301 1
2016-06-05 16:16:32.635124 chunk : 1 (10000000, 5)
2016-06-05 16:16:32.635124 1
2016-06-05 16:17:12.123193 chunk : 2 (10000000, 5)
2016-06-05 16:17:12.123193 1
2016-06-05 16:17:33.376702 chunk : 3 (7670293, 5)
2016-06-05 16:17:33.376702 1
(1198786, 3)
2016-06-05 16:17:34.751060 Done 


In [42]:
grps.groupby('is_booking').count().reset_index().head(10)
len(grps)

1198786

In [12]:
grps[grps.is_booking<20].is_booking.hist(bins=100)

In [10]:
grps20l = grps[grps.is_booking<=20][['user_id']]; grps20l.head()
grps20g = grps[grps.is_booking>20][['user_id']]; 
grps20l.shape

(1184100, 1)

In [17]:
def rxEstimate (rx,hcl) :
    rxx = np.zeros(rx.shape[1],dtype=int)
    for i in range(rx.shape[1]) :
        xx = (hcl==pd.Series(rx[:,i])) ; xx=xx.groupby(by=xx).count(); 
        try     : rxx[i] = xx[True]
        except  : rxx[i] = 0 
    return(rxx)
def estimate (rxx) : 
    res = 0.0
    for i in range(rxx.shape[0]) : res += rxx[i]/(i+1)
    return(res)

In [19]:
# Validation on train(is_booking==1) datas
Validation = True
if Validation :        
    debugV  = True
    debugV  = False
    
    print(datetime.datetime.now(),'Validation',' debug=',debugV)
    #readerTrain = pd.read_csv(fileValidation,chunksize=chunkSize);
    
    chunkSizeV   = 1000000
    if debugV : chunkSizeV   = 10000
    
    readerTrain = pd.read_csv(fileValidation,chunksize=chunkSizeV,usecols=ruleValidate+['user_id']);
    #readerTrain
    
    result = 0.0; resultl = []
    
    r1xx   = np.zeros((nTop),dtype=int)
    r2xx   = np.zeros((nTop),dtype=int)
    r3xx   = np.zeros((nTop),dtype=int)
    r4xx   = np.zeros((nTop),dtype=int)
    r3bxx  = np.zeros((nTop),dtype=int)
    
    nEvent = 0;  
    ichunk = -1
    
    for chunk in readerTrain :
        ichunk += 1
        irow = -1;
        if (ichunk>1) and debugV : print(datetime.datetime.now(),'break'); break
        print(datetime.datetime.now(),'chunk :',ichunk,chunk.shape)
        
        # for round estimate (EFFECT IS NEGATIVE)
        #chunk['orig_destination_distance']=round(chunk['orig_destination_distance'],-1);
        
        # Estimate influence for other user groups
        #chunk   = chunk.merge(grps20l,how='inner',on=['user_id'])
        
        # ------------------------------------------------------
        chunk   = chunk.merge(rule1Test[0],how='left',on=rule1,suffixes=["_0","_1"]); #print(chunk.head())
        chunk   = chunk.merge(rule2Test[0],how='left',on=rule2,suffixes=["_1","_2"]); #print(chunk.head())
        chunk   = chunk.merge(rule3Test[0],how='left',on=rule3,suffixes=["_2","_3"]); #print(chunk.head())
        chunk   = chunk.merge(rule4Test[0],how='left',on=rule4,suffixes=["_3","_4"]); #print(chunk.head())
        chunk   = chunk.merge(rule4Test[0],how='left',on=rule4,suffixes=["_4","_3b"]); #print(chunk.head())
        
        chunk.rename(columns={'nn':'nn_3b'},inplace=True)
        
        rz  = np.array([-1]*nTop)
        
        rr  = chunk['nn_1'].copy(); rr[chunk.nn_1.isnull()] = 0; rr=rr.astype(int); rr1=rr
        #print(rr.values[0:5],chunk.nn_1.isnull().tolist()[0:5])
        r1x = rule1Arr5[rr.values];
        r1x[chunk.nn_1.isnull().values] = rz;
        
        rr  = chunk['nn_2'].copy(); rr[chunk.nn_2.isnull()] = 0; rr=rr.astype(int); rr2=rr
        r2x = rule2Arr5[rr.values];
        r2x[chunk.nn_2.isnull().values] = rz;
        
        rr  = chunk['nn_3'].copy(); rr[chunk.nn_3.isnull()] = 0; rr=rr.astype(int); rr3=rr
        r3x = rule3Arr5[rr.values];
        r3x[chunk.nn_3.isnull().values] = rz;
        
        rr  = chunk['nn_4'].copy(); rr[chunk.nn_4.isnull()] = 0; rr=rr.astype(int); rr4=rr
        r4x = rule4Arr5[rr.values];
        r4x[chunk.nn_4.isnull().values] = rz;
        
        #---------------------------- add rule 3a
        
        rr  = chunk['nn_3b'].copy(); rr[chunk.nn_3b.isnull()] = 0; rr=rr.astype(int); rr3b=rr
        r3bx = rule3bArr5[rr.values];
        r3bx[chunk.nn_3b.isnull().values] = rz;
        
        hcl = chunk['hotel_cluster']
        
        
        r1xx += rxEstimate(r1x,hcl)
        r2xx += rxEstimate(r2x,hcl)
        r3xx += rxEstimate(r3x,hcl)
        r4xx += rxEstimate(r4x,hcl)
        r3bxx+= rxEstimate(r3bx,hcl)
        
        for irow in range(chunk.shape[0]):
            #if (irow>25000) : break;
            ra, rb  = [],[]
            nEvent += 1;
            xEvent  = 0;
            
            rrr1 = rule1Test[1][rr1[irow]]
            rrr2 = rule1Test[1][rr2[irow]]
            
            rr   = r1x[irow].tolist()+r2x[irow].tolist()+r3x[irow].tolist()+r4x[irow].tolist();
            #rr   = r2x[irow].tolist()+r1x[irow].tolist()+r3x[irow].tolist()+r4x[irow].tolist();
            res5 = [i for i in rr if i>-1][0:nTop]
                
            res = 0.0
            if hcl[irow] in res5 : res = 1/(res5.index(hcl[irow])+1)
            result += res
            resultl.append([res,xEvent])
            
            
            #if (nEvent%10000==0) : print(ichunk,irow,result,result/nEvent,hcl[irow],res5) #,ra,rb)
            
        if (ichunk>=1) and debugV : print(datetime.datetime.now(),'break'); break
        
    print('----> result = ',result,result/nEvent,len(resultl))
    print(datetime.datetime.now(),'Done')
            
            



2016-06-06 09:19:28.180212 Validation  debug= False
2016-06-06 09:19:32.732125 chunk : 0 (1000000, 12)
2016-06-06 09:21:35.608241 chunk : 1 (1000000, 12)
2016-06-06 09:23:37.499446 chunk : 2 (1000000, 12)
2016-06-06 09:25:35.613262 chunk : 3 (145036, 12)
----> result =  1093247.0333328506 0.3476103400192718 3145036
2016-06-06 09:25:49.221224 Done


In [28]:
chunk.head()

,dty,user_location_city,orig_destination_distance,user_id,dt0y,dt0m,srch_destination_id,is_booking,cnt,hotel_country,hotel_market,hotel_cluster,nn_1,nn_2,nn_3,nn_4,nn
0,2014,35390,911.5142,93,2014,11,14984,0,1,50,1457,92,866270.0,11511.0,10486.0,47,47
1,2014,10067,NaN,501,2014,8,8267,0,1,50,675,98,NaN,6058.0,5697.0,47,47
2,2014,47725,NaN,1048,2015,6,8803,0,1,151,69,85,NaN,6494.0,6063.0,145,145
3,2014,47725,NaN,1048,2015,6,8803,0,1,151,1236,82,NaN,6497.0,6063.0,145,145
4,2014,47725,NaN,1048,2015,6,8803,0,1,151,1197,29,NaN,6496.0,6063.0,145,145


In [20]:
print(r1xx,r1xx.sum(),r1xx/r1xx.sum(),'\n',r2xx,r2xx.sum(),r2xx/r2xx.sum(),'\n',r3xx,r3xx.sum(),'\n',r4xx,r4xx.sum())
print(r3bxx,r3bxx.sum(),r3bxx/r3bxx.sum())
print(estimate(r1xx)/nEvent)
print(estimate(r2xx)/nEvent)
print(estimate(r3xx)/nEvent)
print(estimate(r4xx)/nEvent)
print(estimate(r3bxx)/nEvent)

[449755  44174   3343    223     16] 497511 [  9.04010163e-01   8.87899966e-02   6.71944942e-03   4.48231295e-04
   3.21600929e-05] 
 [423413 304371 234696 191717 164568] 1318765 [ 0.32106782  0.23080003  0.17796651  0.14537617  0.12478948] 
 [390229 281437 228103 179475 158187] 1237431 
 [116970 102491 124696  87057  76522] 507736
[45980 56797 50872 54399 64965] 273013 [ 0.16841689  0.20803771  0.18633545  0.19925425  0.2379557 ]
0.150400594249
0.23359775532
0.217322785282
0.0784886034161
0.0374967355965


In [27]:
import matplotlib
from matplotlib import pyplot as plt
%matplotlib notebook

In [ ]:
resultl
res=pd.DataFrame(resultl); res.columns=['result','who']; res['cumres']=res['result'].cumsum()
#res['who'].hist()
#res['result'].hist()
#res['cumres'].plot()
res[['cumres']].plot()
#res.shape


In [22]:
Submission = True
if Submission :        
    print(datetime.datetime.now(),'Build result')
    #readerTrain = pd.read_csv(fileValidation,chunksize=chunkSize);
    readerTrain = pd.read_csv(fileTest,chunksize=1000000,usecols=ruleTest);
    #readerTrain
    
    result = []
    id     = []
    nEvent = 0;  
    ichunk = -1
    
    r5x    = rule5Arr5
    
    for chunk in readerTrain :
        ichunk += 1
        irow = -1;
        #if (ichunk>3) : print(datetime.datetime.now(),'break'); break
        print(datetime.datetime.now(),'chunk :',ichunk,chunk.shape)
        
        # for round estimate (EFFECT IS NEGATIVE?)
        #chunk['orig_destination_distance']=round(chunk['orig_destination_distance'],-1);
        
        chunk   = chunk.merge(rule1Test[0],how='left',on=rule1,suffixes=["_0","_1"]); #print(chunk.head())
        chunk   = chunk.merge(rule2Test[0],how='left',on=rule2,suffixes=["_1","_2"]); #print(chunk.head())
        chunk   = chunk.merge(rule3Test[0],how='left',on=rule3,suffixes=["_2","_3"]); #print(chunk.head())
        chunk   = chunk.merge(rule4Test[0],how='left',on=rule4,suffixes=["_3","_4"]); #print(chunk.head())
        
        rz  = np.array([-1]*nTop)
        
        rr  = chunk['nn_1'].copy(); rr[chunk.nn_1.isnull()] = 0; rr=rr.astype(int); 
        #print(rr.values[0:5],chunk.nn_1.isnull().tolist()[0:5])
        r1x = rule1Arr5[rr.values];
        r1x[chunk.nn_1.isnull().values] = rz;
        
        rr  = chunk['nn_2'].copy(); rr[chunk.nn_2.isnull()] = 0; rr=rr.astype(int); #print(rr.values[0:5])
        r2x = rule2Arr5[rr.values];
        r2x[chunk.nn_2.isnull().values] = rz;
        
        rr  = chunk['nn_3'].copy(); rr[chunk.nn_3.isnull()] = 0; rr=rr.astype(int); #print(rr.values[0:5])
        r3x = rule3Arr5[rr.values];
        r3x[chunk.nn_3.isnull().values] = rz;
        
        rr  = chunk['nn_4'].copy(); rr[chunk.nn_4.isnull()] = 0; rr=rr.astype(int); #print(rr.values[0:5])
        r4x = rule4Arr5[rr.values];
        r4x[chunk.nn_4.isnull().values] = rz;
        
        id= id+chunk['id'].tolist()
        
        for irow in range(chunk.shape[0]):
            #if (irow>25000) : break;
            nEvent += 1;
            
            res5 =        [i for i in r1x[irow] if (i>-1)]
            res5 = res5 + [i for i in r2x[irow] if (i>-1) and (i not in res5)]
            res5 = res5 + [i for i in r3x[irow] if (i>-1) and (i not in res5)]
            res5 = res5 + [i for i in r4x[irow] if (i>-1) and (i not in res5)]
            res5 = res5 + [i for i in r5x       if (i>-1) and (i not in res5)]
            
            
                
            result.append(res5[0:nTop])
            
            #if (nEvent%10000==0) : print(ichunk,irow,result,result/nEvent,hcl[irow],res5) #,ra,rb)
        
    result = [x.__str__().replace(',','')[1:-1] for x in result];
    print('----> result = ',id[0:5],result[0:5],len(result))
    print(datetime.datetime.now(),'Done')
    
    
            

2016-06-06 09:45:00.190060 Build result
2016-06-06 09:45:05.652017 chunk : 0 (1000000, 9)
2016-06-06 09:46:57.170427 chunk : 1 (1000000, 9)
2016-06-06 09:48:52.526702 chunk : 2 (528243, 9)
----> result =  [0, 1, 2, 3, 4] ['5 37 55 22 11', '5 35 37 41 22', '91 31 96 77 48', '1 50 51 54 10', '50 51 91 2 59'] 2528243
2016-06-06 09:50:43.501187 Done


In [23]:
testResult = pd.DataFrame({'id':id,'hotel_cluster':result})
fileResult = os.path.join(dirResult,'df-Leak-'+datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S')+'.csv'); fileResult

'..\\Result\\Leak\\df-Leak-2016-06-06-09-51-24.csv'

In [24]:
print(testResult.head(10),testResult.tail(10))

    hotel_cluster  id
0   5 37 55 22 11   0
1   5 35 37 41 22   1
2  91 31 96 77 48   2
3   1 50 51 54 10   3
4   50 51 91 2 59   4
5  91 42 95 48 39   5
6   95 21 98 91 2   6
7  41 59 21 37 28   7
8  88 50 51 54 10   8
9  55 32 34 10 50   9           hotel_cluster       id
2528233   21 59 95 4 19  2528233
2528234  59 87 65 81 52  2528234
2528235   82 85 36 7 78  2528235
2528236   85 7 93 48 15  2528236
2528237  71 80 84 92 90  2528237
2528238   34 26 73 0 84  2528238
2528239  57 62 46 36 82  2528239
2528240  54 50 51 10 94  2528240
2528241  50 47 43 15 32  2528241
2528242  12 36 57 62 81  2528242


In [25]:
print(testResult[['id','hotel_cluster']].head())
testResult[['hotel_cluster']].to_csv(fileResult,header=True,index_label="id")

   id   hotel_cluster
0   0   5 37 55 22 11
1   1   5 35 37 41 22
2   2  91 31 96 77 48
3   3   1 50 51 54 10
4   4   50 51 91 2 59


In [ ]:
rule1Test[0].head()

In [ ]:
xx=pd.DataFrame(rule1Arr5)
xx['r0'] =rule1Test[0]['user_location_city']
xx['r1'] =rule1Test[0]['orig_destination_distance']
xx['r1x']=round(rule1Test[0]['orig_destination_distance'])
xx.head()
len(xx.groupby(['r0','r1x'])); len(xx)

In [ ]:
xx.columns
xx[['r1',0]].query('r1>11000').plot(kind='scatter',x='r1',y=0)

In [ ]:
# Build round version ODD variable
rule1Test0 = rule1Test[0].copy();
rule1Test0['orig_destination_distance']=round(rule1Test0['orig_destination_distance'],-1);
yy = rule1Test0.groupby(['user_location_city','orig_destination_distance']).count().reset_index(); yy.nn=yy.index; 
print(yy.shape,'\n',yy.head())

xx = rule1Test0.merge(yy,how='left',on=rule1,suffixes=['_l','_r']); print(len(xx))

rule1Test0 = yy
rule1Test1 = np.zeros((len(yy),nClusters),dtype=int); 
for i in range(len(xx)) : rule1Test1[xx.nn_r[i]] += rule1Test[1][xx.nn_l[i]]
    
print(rule1Test[1][0])
print(rule1Test1[0])

In [ ]:
rule1ArrX = np.apply_along_axis(findTop,1,rule1Test1); print(rule1ArrX.shape,'\n',rule1ArrX[0:5]);

In [ ]:
# Change on round version
rule1Test[0]=rule1Test0
rule1Test[1]=rule1Test1
rule1Arr5=rule1ArrX


In [ ]:
 print(rule1Test0.shape,rule1Test1.shape,rule1ArrX.shape,'\n',rule1ArrX[0:5]);

In [ ]:
rule1ArrX[0:10]; pd.Series(rule1ArrX[:,4]).groupby(rule1ArrX[:,0]).count()

In [ ]:
pd.DataFrame(rule1ArrX).hist(bins=40)